<a href="https://colab.research.google.com/github/MarkLo127/stockinfo/blob/main/datatest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=13f9850ea7fe05b776fa25b8d6539e0252114e389531f60046bcb2da2a6d6c3b
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [2]:
pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.8 MB/s eta 0:00:00


In [3]:
pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.6 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=957e13c78992490233b1289244591fcb44f6b2394e6d91fd696159db0db1a46d
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:


In [4]:
# 系統相關與時間操作
from posixpath import expanduser
from datetime import datetime, timedelta
import time

# 資料分析
import pandas as pd  # 資料處理
import json  # JSON 處理

# 資料擷取與網路相關
import yfinance as yf  # 股票數據
import requests as res  # HTTP 請求
from bs4 import BeautifulSoup  # 網頁解析
import re  # 正規表示式處理

# 地理與地圖相關
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderInsufficientPrivileges
from IPython.display import display, IFrame
import folium

# 畫圖相關
import plotly
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px

# 財務技術分析
from ta.trend import MACD  # MACD 技術指標
from ta.momentum import StochasticOscillator, RSIIndicator  # 隨機震盪與 RSI 技術指標

#翻譯
from deep_translator import GoogleTranslator
import concurrent.futures

1.大盤指數

In [5]:
class plotindex:
    def __init__(self, period, time, plot_type='index'):
        self.period = period
        self.time = time
        self.plot_type = plot_type
        self.symbols = {
            'index': ['^IXIC', '^VIX', '^GSPC', '^DJI', '^SOX', '^RUT'],
            'foreign': ['^GSPC', '^IXIC', '^HSI', '399001.SZ', '^TWII', '^N225']
        }
        self.data = {}

    def fetch_data(self):
        """Fetch historical data for the selected indexes."""
        for symbol in self.symbols[self.plot_type]:
            self.data[symbol] = yf.download(symbol, period=self.period)['Close']

    def plot_index(self):
        """Plot the US indexes."""
        # Create Plotly subplot figure for indexes
        fig = make_subplots(rows=3, cols=2, subplot_titles=["NASDAQ", "VIX", "S&P 500", "DJIA", "PHLX Semiconductor Sector", "Russell-2000"])

        for i, symbol in enumerate(self.symbols['index']):
            fig.add_trace(go.Scatter(x=self.data[symbol].index, y=self.data[symbol].values, mode='lines', name=symbol), row=(i // 2) + 1, col=(i % 2) + 1)

        fig.update_layout(height=1000, width=1000, showlegend=False)
        return fig

    def plot_foreign(self):
        """Plot the foreign indexes."""
        # Create Plotly subplot figure for foreign indexes
        fig = make_subplots(rows=3, cols=2, subplot_titles=["S&P 500", "NASDAQ", "恆生指數", "深證指數", "加權指數", "日經指數"])

        for i, symbol in enumerate(self.symbols['foreign']):
            row = (i // 2) + 1
            col = (i % 2) + 1

            # Apply conversion for foreign indexes
            if symbol in ['^HSI', '399001.SZ', '^TWII', '^N225']:
                conversion_factor = {'^HSI': 0.1382, '399001.SZ': 0.1382, '^TWII': 0.0308, '^N225': 0.0064}
                fig.add_trace(go.Scatter(x=self.data[symbol].index, y=(self.data[symbol] * conversion_factor[symbol]).values, mode='lines', name=symbol), row=row, col=col)
            else:
                fig.add_trace(go.Scatter(x=self.data[symbol].index, y=self.data[symbol].values, mode='lines', name=symbol), row=row, col=col)

        fig.update_layout(height=1000, width=1000, showlegend=False)
        return fig

    def plot(self):
        """Plot the financial data based on the selected type."""
        self.fetch_data()
        if self.plot_type == 'index':
            fig = self.plot_index()
        elif self.plot_type == 'foreign':
            fig = self.plot_foreign()
        fig.show()

period = '1y'
time = '一年'
plt = plotindex(period, time, plot_type='index')
plt2 = plotindex(period, time, plot_type='foreign')

# 繪製圖表
plt.plot()
plt2.plot()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


2.公司基本資訊

In [6]:
class cominfo:
    tran_dict = {
        'address1': '地址',
        'city': '城市',
        'country': '國家',
        'phone': '電話',
        'website': '網站',
        'industry': '行業',
        'sector': '產業',
        'longBusinessSummary': '公司簡介',
        'fullTimeEmployees': '全職員工數量',
        'marketCap': '市值',
        'totalRevenue': '總收入',
        'netIncomeToCommon': '淨利潤',
        'trailingEps': '每股盈餘(EPS)',
        'trailingPE': '本益比(PE)',
        'dividendRate': '股息率',
        'dividendYield': '股息殖利率',
        'beta': 'Beta值',
        'profitMargins': '利潤率',
        'revenueGrowth': '收入成長率',
        'earningsGrowth': '收益成長率'
    }

    def __init__(self, symbol):
        self.symbol = symbol
        self.com_info = self.get_com_info()

    def get_com_info(self):
        """獲取公司的詳細資訊"""
        stock = yf.Ticker(self.symbol)
        com_info = stock.info

        # 儲存資訊為 JSON
        with open("df.json", "w") as outfile:
            json.dump(com_info, outfile)

        # 讀取資料並轉置
        df = pd.read_json("df.json").head(1).transpose()
        return df

    def tran_info(self):
        """翻譯公司資訊並格式化顯示"""
        tran_info = {}
        self.com_info.index = self.com_info.index.str.strip()  # 去除索引空格

        for key in self.tran_dict.keys():
            if key in self.com_info.index:
                value = self.com_info.loc[key].values[0]
                if isinstance(value, float):
                    if 'rate' in key or 'Growth' in key or 'Yield' in key:
                        value = f"{value * 100:.2f}%"  # 百分比格式
                    else:
                        value = f"{value:,.2f}"  # 千分位格式
                elif isinstance(value, int):
                    value = f"{value:,}"  # 千分位格式
                tran_info[self.tran_dict[key]] = value

        return pd.DataFrame.from_dict(tran_info, orient='index', columns=['內容'])

    def get_location(self, address, city, country):
        """獲取公司位置的經緯度，若 address 無法定位則使用 city 定位"""
        geolocator = Nominatim(user_agent="company_info_app")

        # 優先嘗試使用 address 定位
        location = geolocator.geocode(f"{address}, {city}, {country}")

        # 如果 address 無法定位，則使用 city 和 country 來定位
        if location is None:
            location = geolocator.geocode(f"{city}, {country}")
        return location

    def display_map(self, location, tran_df):
        """顯示公司位置的地圖"""
        m = folium.Map(location=[location.latitude, location.longitude], zoom_start=13)
        folium.Marker([location.latitude, location.longitude],
                      popup=tran_df.to_html(escape=False),
                      tooltip=f'{self.symbol}位置').add_to(m)

        # 直接在Notebook中顯示地圖
        display(m)

# 主程式
symbol = 'NVDA'  # 替換成想查詢的公司代碼
company = cominfo(symbol)

# 獲取公司資訊
tran_df = company.tran_info()
print(tran_df)

# 獲取地理位置並顯示地圖
location = company.get_location(company.com_info.loc['address1'].values[0],
                                 company.com_info.loc['city'].values[0],
                                 company.com_info.loc['country'].values[0])

if location:
    company.display_map(location, tran_df)
else:
    print("無法定位該公司的地理位置。")

                                                          內容
地址                                 2788 San Tomas Expressway
城市                                               Santa Clara
國家                                             United States
電話                                              408 486 2000
網站                                    https://www.nvidia.com
行業                                            Semiconductors
產業                                                Technology
公司簡介       NVIDIA Corporation provides graphics and compu...
全職員工數量                                                29,600
市值                                         3,013,510,430,720
總收入                                           96,307,003,392
淨利潤                                           53,007,998,976
每股盈餘(EPS)                                               2.14
本益比(PE)                                                57.41
股息率                                                     0.04
股息殖利率                   

3.公司經營狀況



In [7]:
class com_mange:
    def __init__(self, symbol):
        self.symbol = symbol
        self.data = {}
        self.url = f"https://finviz.com/quote.ashx?t={symbol}&p=d#statements"
        self.headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

    def fetch_data(self):
        try:
            response = res.get(self.url, headers=self.headers)
            response.raise_for_status()
        except res.exceptions.RequestException as e:
            print(f"獲取 {self.symbol} 數據時出錯: {e}")
            return None

        # Parse the HTML to get the data
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', class_='snapshot-table2')
        if not table:
            print("頁面上未找到表格")
            return None

        # Extract data into a dictionary
        rows = table.find_all('tr')
        for row in rows:
            cells = row.find_all('td')
            for i in range(0, len(cells), 2):
                key = cells[i].get_text(strip=True)
                value = cells[i + 1].get_text(strip=True)
                self.data[key] = value

        # Print the extracted table data
        for key, value in self.data.items():
            print(f"{key}: {value}")

# Example usage
symbol = 'NVDA'  # You can change the stock symbol here
mange_info = com_mange(symbol)
mange_info.fetch_data()

Index: NDX, S&P 500
P/E: 57.68
EPS (ttm): 2.13
Insider Own: 3.98%
Shs Outstand: 24.64B
Perf Week: -0.96%
Market Cap: 3013.51B
Forward P/E: 30.43
EPS next Y: 42.18%
Insider Trans: -1.05%
Shs Float: 23.55B
Perf Month: 15.67%
Income: 53.01B
PEG: 1.10
EPS next Q: 0.74
Inst Own: 65.60%
Short Float: 1.17%
Perf Quarter: -2.37%
Sales: 96.31B
P/S: 31.29
EPS this Y: 119.08%
Inst Trans: -0.09%
Short Ratio: 0.84
Perf Half Y: 43.01%
Book/sh: 2.37
P/B: 51.88
ROA: 78.66%
Short Interest: 275.33M
Perf Year: 182.30%
Cash/sh: 1.42
P/C: 86.60
EPS next 5Y: 52.53%
ROE: 123.77%
52W Range: 39.23 - 140.76
Perf YTD: 148.07%
Dividend Est.: 0.04 (0.03%)
P/FCF: 64.41
EPS past 5Y: 48.40%
ROI: 78.04%
52W High: -12.72%
Beta: 1.67
Dividend TTM: 0.10 (0.08%)
Quick Ratio: 3.79
Sales past 5Y: 46.68%
Gross Margin: 75.98%
52W Low: 213.15%
ATR (14): 5.32
Dividend Ex-Date: Sep 12, 2024
Current Ratio: 4.27
EPS Y/Y TTM: 415.32%
Oper. Margin: 61.87%
RSI (14): 56.11
Volatility: 3.92% 4.04%
Employees: 29600
Debt/Eq: 0.17
Sales Y/

4.公司財報

In [8]:
class financialrrport_y:
    def __init__(self, symbol, target_language='zh-TW'):
        self.symbol = symbol
        self.target_language = target_language
        self.balance_sheet = None
        self.income_statement = None
        self.cash_flow = None

    # 多進程翻譯函數
    def tran(self, texts):
        tran = GoogleTranslator(source='en', target=self.target_language)
        # 使用多進程進行批量翻譯
        with concurrent.futures.ThreadPoolExecutor() as executor:
            translations = list(executor.map(tran.translate, texts))
        return translations

    # 使用多進程翻譯DataFrame的列名和索引名
    def tran_col(self, df):
        if df is not None and not df.empty:
            # 多進程翻譯列名
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = pd.MultiIndex.from_tuples(
                    [tuple(self.tran([str(col) for col in level])) for level in df.columns.levels]
                )
            else:
                df.columns = self.tran([str(col) for col in df.columns])

            # 多進程翻譯索引名
            if isinstance(df.index, pd.MultiIndex):
                df.index = pd.MultiIndex.from_tuples(
                    [tuple(self.tran([str(idx) for idx in level])) for level in df.index.levels]
                )
            else:
                df.index = self.tran([str(idx) for idx in df.index])

        return df

    # 獲取財務報表
    def get_financial(self):
        try:
            stock = yf.Ticker(self.symbol)
            self.balance_sheet = stock.balance_sheet
            self.income_statement = stock.financials
            self.cash_flow = stock.cashflow
        except Exception as e:
            print(f"獲取財務報表發生錯誤：{str(e)}")
            self.balance_sheet, self.income_statement, self.cash_flow = None, None, None

    # 翻譯財務報表
    def tran_financial(self):
        if self.balance_sheet is not None:
            self.balance_sheet = self.tran_col(self.balance_sheet)
        if self.income_statement is not None:
            self.income_statement = self.tran_col(self.income_statement)
        if self.cash_flow is not None:
            self.cash_flow = self.tran_col(self.cash_flow)

    # 顯示財務報表
    def display_financial(self):
        if self.balance_sheet is not None:
            print(f"{self.symbol}-資產負債表/年")
            print(self.balance_sheet)

        if self.income_statement is not None:
            print(f"{self.symbol}-綜合損益表/年")
            print(self.income_statement)

        if self.cash_flow is not None:
            print(f"{self.symbol}-現金流量表/年")
            print(self.cash_flow)

# 主程式執行
symbol = 'NVDA'
tran = financialrrport_y(symbol)

# 獲取並翻譯財務報表
tran.get_financial()
tran.tran_financial()

# 顯示翻譯後的報表
tran.display_financial()

NVDA-資產負債表/年
              2024-01-31 00:00:00 2023-01-31 00:00:00 2022-01-31 00:00:00  \
庫存股數量                         NaN                 NaN                 NaN   
普通股數量               24640000000.0       24661365720.0       25060000000.0   
已發行股份               24640000000.0       24661365720.0       25060000000.0   
淨債務                  2429000000.0        7564000000.0        8956000000.0   
總債務                 11056000000.0       12031000000.0       11831000000.0   
...                           ...                 ...                 ...   
應收帳款總額                        NaN                 NaN                 NaN   
應收帳款總額                        NaN                 NaN                 NaN   
現金 現金等價物與短期投資       25984000000.0       13296000000.0       21208000000.0   
現金及現金等價物            18704000000.0        9907000000.0       19218000000.0   
現金及現金等價物             7280000000.0        3389000000.0        1990000000.0   

              2021-01-31 00:00:00 2020-01-31 00:00:00  
庫存股數量 

In [9]:
class financialrrport_q:
    def __init__(self, symbol, target_language='zh-TW'):
        self.symbol = symbol
        self.target_language = target_language
        self.quarterly_balance_sheet = None
        self.quarterly_income_statement = None
        self.quarterly_cash_flow = None

    # 多進程翻譯函數
    def tran(self, texts):
        tran = GoogleTranslator(source='en', target=self.target_language)
        # 使用多進程進行批量翻譯
        with concurrent.futures.ThreadPoolExecutor() as executor:
            translations = list(executor.map(tran.translate, texts))
        return translations

    # 使用多進程翻譯DataFrame的列名和索引名
    def tran_col(self, df):
        if df is not None and not df.empty:
            # 多進程翻譯列名
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = pd.MultiIndex.from_tuples(
                    [tuple(self.tran([str(col) for col in level])) for level in df.columns.levels]
                )
            else:
                df.columns = self.tran([str(col) for col in df.columns])

            # 多進程翻譯索引名
            if isinstance(df.index, pd.MultiIndex):
                df.index = pd.MultiIndex.from_tuples(
                    [tuple(self.tran([str(idx) for idx in level])) for level in df.index.levels]
                )
            else:
                df.index = self.tran([str(idx) for idx in df.index])

        return df

    # 獲取季度財務報表
    def get_financial_q(self):
        try:
            stock = yf.Ticker(self.symbol)
            self.quarterly_balance_sheet = stock.quarterly_balance_sheet
            self.quarterly_income_statement = stock.quarterly_financials
            self.quarterly_cash_flow = stock.quarterly_cashflow
        except Exception as e:
            print(f"獲取財務報表發生錯誤：{str(e)}")
            self.quarterly_balance_sheet, self.quarterly_income_statement, self.quarterly_cash_flow = None, None, None

    # 翻譯季度財務報表
    def tran_financial_q(self):
        if self.quarterly_balance_sheet is not None:
            self.quarterly_balance_sheet = self.tran_col(self.quarterly_balance_sheet)
        if self.quarterly_income_statement is not None:
            self.quarterly_income_statement = self.tran_col(self.quarterly_income_statement)
        if self.quarterly_cash_flow is not None:
            self.quarterly_cash_flow = self.tran_col(self.quarterly_cash_flow)

    # 顯示季度財務報表
    def display_financial_q(self):
        if self.quarterly_balance_sheet is not None:
            print(f"{self.symbol}-資產負債表/季")
            print(self.quarterly_balance_sheet)

        if self.quarterly_income_statement is not None:
            print(f"{self.symbol}-綜合損益表/季")
            print(self.quarterly_income_statement)

        if self.quarterly_cash_flow is not None:
            print(f"{self.symbol}-現金流量表/季")
            print(self.quarterly_cash_flow)

# 主程式執行
symbol = 'NVDA'
tran_q = financialrrport_q(symbol)

# 獲取並翻譯季度財務報表
tran_q.get_financial_q()
tran_q.tran_financial_q()

# 顯示翻譯後的季度報表
tran_q.display_financial_q()

NVDA-資產負債表/季
              2024-07-31 00:00:00 2024-04-30 00:00:00 2024-01-31 00:00:00  \
庫存股數量                         NaN                 NaN                 NaN   
普通股數量               24562000000.0       24598341970.0       24640000000.0   
已發行股份               24562000000.0       24598341970.0       24640000000.0   
淨債務                           NaN        2123000000.0        2429000000.0   
總債務                 10015000000.0       11237000000.0       11056000000.0   
...                           ...                 ...                 ...   
應收帳款                14132000000.0       12365000000.0        9999000000.0   
應收帳款                14132000000.0       12365000000.0        9999000000.0   
現金 現金等價物與短期投資       34800000000.0       31438000000.0       25984000000.0   
其他短期投資              26237000000.0       23851000000.0       18704000000.0   
現金及現金等價物             8563000000.0        7587000000.0        7280000000.0   

              2023-10-31 00:00:00 2023-07-31 00:00:00 2023-04-

5.交易數據

In [10]:
class tradedata:

    @staticmethod
    def getdata(symbol, time_range):
        stock_data = yf.download(symbol, period=time_range)
        return stock_data

    @staticmethod
    def getdata_timerange(symbol,start,end):
        stock_data = yf.download(symbol,start=start,end=end)
        return stock_data

    @staticmethod
    def calculate_difference(stock_data, period_days):
        latest_price = stock_data.iloc[-1]["Adj Close"]
        previous_price = stock_data.iloc[-period_days]["Adj Close"] if len(stock_data) > period_days else stock_data.iloc[0]["Adj Close"]
        price_difference = latest_price - previous_price
        percent_difference = (price_difference / previous_price) * 100
        return price_difference, percent_difference

    @staticmethod
    def plot_kline(stock_data):
        fig = plotly.subplots.make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.01, row_heights=[0.8, 0.5, 0.5, 0.5])

        mav5 = stock_data['Adj Close'].rolling(window=5).mean()
        mav20 = stock_data['Adj Close'].rolling(window=20).mean()
        mav60 = stock_data['Adj Close'].rolling(window=60).mean()

        rsi = RSIIndicator(close=stock_data['Adj Close'], window=14)
        macd = MACD(close=stock_data['Adj Close'], window_slow=26, window_fast=12, window_sign=9)

        fig.add_trace(go.Candlestick(x=stock_data.index, open=stock_data['Open'], high=stock_data['High'], low=stock_data['Low'], close=stock_data['Adj Close']), row=1, col=1)
        fig.update_layout(xaxis_rangeslider_visible=False)

        fig.add_trace(go.Scatter(x=stock_data.index, y=mav5, line=dict(color='blue', width=2), name='MAV-5', showlegend=False), row=1, col=1)
        fig.add_trace(go.Scatter(x=stock_data.index, y=mav20, line=dict(color='orange', width=2), name='MAV-20', showlegend=False), row=1, col=1)
        fig.add_trace(go.Scatter(x=stock_data.index, y=mav60, line=dict(color='purple', width=2), name='MAV-60', showlegend=False), row=1, col=1)

        colors = ['green' if row['Open'] - row['Adj Close'] >= 0 else 'red' for _, row in stock_data.iterrows()]
        fig.add_trace(go.Bar(x=stock_data.index, y=stock_data['Volume'], marker_color=colors, name='Volume', showlegend=False), row=2, col=1)

        fig.add_trace(go.Scatter(x=stock_data.index, y=rsi.rsi(), line=dict(color='purple', width=2), showlegend=False), row=3, col=1)
        fig.add_trace(go.Scatter(x=stock_data.index, y=[70]*len(stock_data.index), line=dict(color='red', width=1), name='Overbought', showlegend=False), row=3, col=1)
        fig.add_trace(go.Scatter(x=stock_data.index, y=[30]*len(stock_data.index), line=dict(color='green', width=1), name='Oversold', showlegend=False), row=3, col=1)

        colorsM = ['green' if val >= 0 else 'red' for val in macd.macd_diff()]
        fig.add_trace(go.Bar(x=stock_data.index, y=macd.macd_diff(), marker_color=colorsM, showlegend=False), row=4, col=1)
        fig.add_trace(go.Scatter(x=stock_data.index, y=macd.macd(), line=dict(color='orange', width=2), showlegend=False), row=4, col=1)
        fig.add_trace(go.Scatter(x=stock_data.index, y=macd.macd_signal(), line=dict(color='blue', width=1), showlegend=False), row=4, col=1)

        fig.update_yaxes(title_text="Price", row=1, col=1)
        fig.update_yaxes(title_text="Volume", row=2, col=1)
        fig.update_yaxes(title_text="RSI", row=3, col=1)
        fig.update_yaxes(title_text="MACD", row=4, col=1)

        fig.update_layout(showlegend=False)
        fig.show()


options = input('1、2')

if options == '1':
    stock = input('輸入股票代號')
    time_range = '1y'
    data = tradedata.getdata(stock, time_range)
    price_diff, perc_diff = tradedata.calculate_difference(data, 30)
    print(f"Price Difference: {price_diff}, Percentage Difference: {perc_diff}%")
    tradedata.plot_kline(data)
elif options == '2':
    stock = input('輸入股票代號')
    start = input('輸入開始日期')
    end = input('輸入結束日期')
    data = tradedata.getdata_timerange(stock, start, end)
    price_diff, perc_diff = tradedata.calculate_difference(data, 30)
    print(f"Price Difference: {price_diff}, Percentage Difference: {perc_diff}%")
    tradedata.plot_kline(data)
else:
    print('輸入錯誤')

1、21
輸入股票代號NVDA


[*********************100%***********************]  1 of 1 completed


Price Difference: -0.8794174194335938, Percentage Difference: -0.7107585638245985%


6.期權

In [11]:
class Option:
    def __init__(self, symbol):
        self.symbol = symbol

    def get_option_dates(self):
        stock = yf.Ticker(self.symbol)
        return stock.options

    def options_calls_date(self, date):
        stock = yf.Ticker(self.symbol)
        option_chain = stock.option_chain(date)
        return option_chain.calls

    def options_puts_date(self, date):
        stock = yf.Ticker(self.symbol)
        option_chain = stock.option_chain(date)
        return option_chain.puts

# Example usage
opion = Option('NVDA')

# Get available option dates
option_dates = opion.get_option_dates()
print(pd.DataFrame(option_dates))

# Input a date for option calls and puts
date = input('輸入日期: ')

calls = opion.options_calls_date(date)
puts = opion.options_puts_date(date)

# Print the options data
print('Calls Data:')
print(calls)

print('Puts Data:')
print(puts)

             0
0   2024-10-04
1   2024-10-11
2   2024-10-18
3   2024-10-25
4   2024-11-01
5   2024-11-08
6   2024-11-15
7   2024-12-20
8   2025-01-17
9   2025-02-21
10  2025-03-21
11  2025-04-17
12  2025-05-16
13  2025-06-20
14  2025-08-15
15  2025-09-19
16  2025-12-19
17  2026-01-16
18  2026-06-18
19  2026-12-18
20  2027-01-15
輸入日期: 2024-10-04
Calls Data:
         contractSymbol             lastTradeDate  strike  lastPrice  bid  \
0   NVDA241004C00050000 2024-10-03 19:17:29+00:00    50.0      72.40  0.0   
1   NVDA241004C00055000 2024-10-02 18:57:49+00:00    55.0      63.60  0.0   
2   NVDA241004C00060000 2024-10-03 19:15:29+00:00    60.0      62.40  0.0   
3   NVDA241004C00065000 2024-10-03 14:47:59+00:00    65.0      57.85  0.0   
4   NVDA241004C00067000 2024-09-27 15:40:41+00:00    67.0      53.60  0.0   
..                  ...                       ...     ...        ...  ...   
94  NVDA241004C00185000 2024-10-02 14:52:02+00:00   185.0       0.01  0.0   
95  NVDA241004C00190000 2

7.SEC文件

In [6]:
class SECReport:
    def __init__(self, symbol):
        self.symbol = symbol
        self.stock = yf.Ticker(self.symbol)

    def get_sec_filings(self):
        """
        獲取公司的 SEC 文件列表。
        """
        try:
            return self.stock.sec_filings
        except Exception as e:
            print(f"獲取 SEC 文件時發生錯誤：{str(e)}")
            return None

    def display_filings(self):
        """
        顯示 SEC 文件列表。
        """
        filings = self.stock.sec_filings
        if filings:
            print(f"公司 {self.symbol} 的 SEC 文件:")
            for filing in filings:
                print(filing)
        else:
            print(f"無法獲取公司 {self.symbol} 的 SEC 文件。")


# 主程式執行
symbol = 'NVDA'
sec_report = SECReport(symbol)
sec_report.display_filings()

公司 NVDA 的 SEC 文件:
{'date': datetime.date(2024, 8, 28), 'epochDate': 1724803200, 'type': '8-K', 'title': 'Corporate Changes & Voting Matters', 'edgarUrl': 'https://finance.yahoo.com/sec-filing/NVDA/0001045810-24-000262_1045810', 'exhibits': {'8-K': 'https://cdn.yahoofinance.com/prod/sec-filings/0001045810/000104581024000262/nvda-20240828.htm', 'EXCEL': 'https://s3.amazonaws.com/finance-pri-uw2/sec-filings/0001045810/000104581024000262/Financial_Report.xlsx', 'EX-99.2': 'https://cdn.yahoofinance.com/prod/sec-filings/0001045810/000104581024000262/q2fy25cfocommentary.htm', 'EX-99.1': 'https://cdn.yahoofinance.com/prod/sec-filings/0001045810/000104581024000262/q2fy25pr.htm'}, 'maxAge': 1}
{'date': datetime.date(2024, 7, 18), 'epochDate': 1721260800, 'type': 'SC 13G', 'title': 'Tender Offer/Acquisition Reports', 'edgarUrl': 'https://finance.yahoo.com/sec-filing/NVDA/0001045810-24-000230_1045810', 'exhibits': {'SC 13G': 'https://cdn.yahoofinance.com/prod/sec-filings/0001832483/000104581024000

8.機構評級

In [12]:
class Holding:
    def __init__(self, symbol):
        self.symbol = symbol

    @staticmethod
    def holder(symbol):
        symbol = yf.Ticker(symbol)
        holders = symbol.institutional_holders

        if holders is not None:
            # 提取 pctHeld, value 和 Holder 欄位，並將持股比例轉換為百分比
            pct_held = holders['pctHeld']  # 正確的列名是 'pctHeld'
            value = holders['Value']
            holder_name = holders['Holder']

            # 動態生成不同顏色列表，每個直條不同顏色
            colors = [f'rgb({i*30 % 255}, {100 + i*40 % 155}, {i*50 % 255})' for i in range(len(holder_name))]

            # 使用 plotly 繪製條形圖
            fig = go.Figure()
            fig.add_trace(go.Bar(
                x=holder_name,
                y=pct_held,
                name='持股比例 (%)',
                marker=dict(color=colors),
                text=value,
                texttemplate='%{text:.2s}',
                textposition='auto'
            ))

            fig.update_layout(
                title=f'機構持股 {symbol.ticker} 比例與價值',
                xaxis_title='機構',
                yaxis_title='持股比例 (%)',
                height=800,
                width=1200
            )
            fig.show()
        else:
            print(f"{symbol} 無機構持有數據")

    @staticmethod
    def fund_holder(symbol):
        symbol = yf.Ticker(symbol)
        funds = symbol.mutualfund_holders

        if funds is not None:
            pct_held = funds['pctHeld']
            value = funds['Value']
            holder_name = funds['Holder']

            colors = [f'rgb({i*35 % 255}, {i*45 % 255}, {150 + i*55 % 105})' for i in range(len(holder_name))]

            fig = go.Figure()
            fig.add_trace(go.Bar(
                x=holder_name,
                y=pct_held,
                name='持股比例 (%)',
                marker=dict(color=colors),
                text=value,
                texttemplate='%{text:.2s}',
                textposition='auto'
            ))

            fig.update_layout(
                title=f'共同基金持股 {symbol.ticker} 比例與價值',
                xaxis_title='共同基金',
                yaxis_title='持股比例 (%)',
                height=800,
                width=1200
            )
            fig.show()
        else:
            print(f"{symbol} 無共同基金持有數據")

    @staticmethod
    def scrape_finviz(symbol):
        url = f"https://finviz.com/quote.ashx?t={symbol}"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        response = res.get(url, headers=headers)

        if response.status_code != 200:
            raise Exception(f"無法從 {url} 獲取數據，狀態碼: {response.status_code}")

        soup = BeautifulSoup(response.content, 'html.parser')

        table = soup.find('table', class_='js-table-ratings styled-table-new is-rounded is-small')

        if table is None:
            raise Exception("未能在頁面上找到評級表格。")

        data = []
        for row in table.find_all('tr')[1:]:
            cols = row.find_all('td')
            data.append({
                "Date": cols[0].text.strip(),
                "Action": cols[1].text.strip(),
                "Analyst": cols[2].text.strip(),
                "Rating Change": cols[3].text.strip(),
                "Price Target Change": cols[4].text.strip() if len(cols) > 4 else None
            })

        df = pd.DataFrame(data)
        df = df.dropna(subset=['Price Target Change'])
        df['Price Target Change'] = df['Price Target Change'].str.replace('→', '->').str.replace(' ', '')

        price_change_ranges = df['Price Target Change'].str.extract(r'\$(\d+)->\$(\d+)')
        price_change_ranges = price_change_ranges.apply(pd.to_numeric)
        df['Price Target Start'] = price_change_ranges[0]
        df['Price Target End'] = price_change_ranges[1]

        rating_order = df['Rating Change'].unique().tolist()
        df['Rating Change'] = pd.Categorical(df['Rating Change'], categories=rating_order, ordered=True)

        fig1 = go.Figure()
        for i, row in df.iterrows():
            fig1.add_trace(go.Scatter(
                x=[row['Price Target Start'], row['Price Target End']],
                y=[row['Analyst'], row['Analyst']],
                mode='lines+markers+text',
                line=dict(color='blue' if row['Price Target End'] >= row['Price Target Start'] else 'red', width=2),
                marker=dict(size=10, color='blue' if row['Price Target End'] >= row['Price Target Start'] else 'red'),
                text=[f"${row['Price Target Start']}", f"${row['Price Target End']}"],
                textposition="top center"
            ))

        fig1.update_layout(
            title=f'機構對 {symbol} 目標價格變化',
            xaxis_title='目標價格',
            yaxis_title='機構',
            yaxis=dict(type='category'),
            showlegend=False,
            height=800,
            width=1200
        )

        fig2 = px.histogram(
            df,
            x='Rating Change',
            title=f'機構評級 {symbol} 變化分佈',
            color='Rating Change',
            category_orders={'Rating Change': rating_order}
        )

        fig2.update_layout(
            height=800,
            width=1200,
            showlegend=False
        )

        fig1.show()
        fig2.show()

# 使用範例
holding = Holding('NVDA')

# 調用 holder 方法
holding.holder(holding.symbol)

# 調用 fund_holder 方法
holding.fund_holder(holding.symbol)

# 調用 scrape_finviz 方法
holding.scrape_finviz(holding.symbol)

9.相關新聞

In [13]:
def getnews(symbol):
    url = f"https://finviz.com/quote.ashx?t={symbol}&p=d"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

    try:
        response = res.get(url, headers=headers)
        response.raise_for_status()  # 確保請求成功
    except res.exceptions.RequestException as e:
        print(f"無法獲取 {symbol} 相關消息: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # 找到所有新聞項目
    news_table = soup.find('table', class_='fullview-news-outer')
    if news_table is None:
        print(f"無法獲取 {symbol} 相關新聞表格")
        return None

    news_items = news_table.find_all('tr')
    news_data = []

    for news_item in news_items:
        cells = news_item.find_all('td')
        if len(cells) < 2:
            continue

        date_info = cells[0].text.strip()
        news_link = cells[1].find('a', class_='tab-link-news')
        if news_link:
            news_title = news_link.text.strip()
            news_url = news_link['href']
            news_data.append({'Date': date_info, 'Title': news_title, 'URL': news_url})

    return news_data

# 使用示例
symbol = 'NVDA'
news_data = getnews(symbol)

if news_data:
    # 將新聞數據轉換為 DataFrame
    df = pd.DataFrame(news_data)
    print(f"{symbol} - 近期相關消息")
    print(df)  # 顯示表格

    # 打印所有新聞連結
    print(f"\n展開 {symbol} - 近期相關消息連結:")
    for news in news_data:
        print(f"**[{news['Title']}]({news['URL']})**")
    print(f"[資料來源](https://finviz.com/quote.ashx?t={symbol})")
else:
    print(f"查無 {symbol} 近期相關消息")

NVDA - 近期相關消息
             Date                                              Title  \
0   Today 06:00AM  2 Magnificent Stocks That Turned $20,000 Into ...   
1         05:47AM  Why Is Wall Street Much More Bullish About Nvi...   
2         05:43AM  What's the Best "Magnificent Seven" Stock to B...   
3         05:35AM  2 Unstoppable Vanguard ETFs to Buy With $500 a...   
4         05:03AM  3 Top Artificial Intelligence (AI) Stocks Read...   
..            ...                                                ...   
95        05:21AM  Nvidia Stock Slips Again. Why Israel-Iran Tens...   
96        05:17AM  Meet the Best-Performing Stock in the Entire S...   
97        05:12AM  Billionaires Are Selling Nvidia Stock and Buyi...   
98        04:41AM  1 Bargain-Basement Stock-Split Stock to Buy Ha...   
99        04:40AM  4 Billion-Dollar Reasons to Buy Nvidia Stock R...   

                                                  URL  
0   https://finance.yahoo.com/m/fb30206a-77f9-3a48...  
1   https